## Holdout Notebook

In [1]:
# import libraries
import pickle
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')


In [2]:
# final_scaler
final_scaler = pickle.load(open('lrscaler.pickle', 'rb'))

# final model
final_model = pickle.load(open('lrmodel.pickle', 'rb'))


In [3]:
# import data
df = pd.read_csv('./data/holdout_data.csv' , index_col=0)


In [4]:
# add column that was dummied that is NOT in holdout set
add_col = ['pay_2_8', 'pay_3_1', 'pay_4_1', 'pay_4_6', 'pay_4_8', 'pay_5_8', 'pay_6_8','pay_2_-2', 'pay_3_-2', 'pay_0_-2', 'pay_4_-2']
for col in add_col:
    df[col] = 0
    

In [5]:
# rename column headers
rename_dict = {'X1':'limit_bal', 'X2':'sex', 'X3':'education', 'X4':'marriage', 'X5':'age', 'X6':'pay_0', 
            'X7':'pay_2','X8':'pay_3', 'X9':'pay_4', 'X10':'pay_5', 'X11':'pay_6', 'X12':'bill_amt1', 
            'X13':'bill_amt2','X14':'bill_amt3', 'X15':'bill_amt4', 'X16':'bill_amt5', 'X17':'bill_amt6', 
            'X18':'pay_amt1','X19':'pay_amt2', 'X20':'pay_amt3', 'X21':'pay_amt4', 'X22':'pay_amt5', 
            'X23':'pay_amt6'}

df = df.rename(columns=rename_dict)


In [6]:
# create list of columns that will be converted to numeric
cols = ['limit_bal','age','bill_amt1', 'bill_amt2', 'bill_amt3',
        'bill_amt4', 'bill_amt5', 'bill_amt6', 'pay_amt1', 'pay_amt2',
        'pay_amt3', 'pay_amt4', 'pay_amt5', 'pay_amt6']

# use .apply method to convert cols to numeric
df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')


In [7]:
# list columns that have large variances
outlier_cols = ['limit_bal','bill_amt1', 'bill_amt2', 'bill_amt3',
        'bill_amt4', 'bill_amt5', 'bill_amt6', 'pay_amt1', 'pay_amt2',
        'pay_amt3', 'pay_amt4', 'pay_amt5', 'pay_amt6']

# loop over columns and anything above or under 6 std coerce them to 6 std respectively
for feat in outlier_cols:
    feat_mean = df[feat].mean()
    feat_std = df[feat].std()
    above_6std = feat_mean + 6 * feat_std
    below_6std = feat_mean - 6 * feat_std
    df[feat] = \
        np.where(df[feat] > above_6std, feat_mean + 6 * feat_std,
                np.where(df[feat] < below_6std, feat_mean - 6 * feat_std, df[feat]))
    

In [8]:
# convert objects to category dtype
objcat = ['sex', 'education', 'marriage', 'pay_0', 'pay_2', 'pay_3', 'pay_4',
       'pay_5', 'pay_6']

for colname in objcat:
    df[colname] = df[colname].astype('category')
    

In [9]:
# set options to display floats with two decimal place
pd.set_option('display.float_format', '{:.2f}'.format)


In [10]:
# read in income demographical dataset
df2 = pd.read_excel('./data/income_sex_age_edu.xls', 
                    sheet_name='125,126', 
                    usecols='C,H', 
                    header=7, 
                    skiprows=2, 
                    nrows=20)


In [11]:
# create new df without na rows
df2 = df2[df2['income'].notna()]


In [12]:
# transpose income df
df2 = df2.T


In [13]:
# assign variable row as header
df2.columns = df2.iloc[1]


In [14]:
# delete row 1
df2.drop(df2.index[1], axis=0, inplace=True)


In [15]:
# strip leading or trailing spaces in column headers
df2.columns = df2.columns.str.strip()


In [16]:
# rename columns
df2.rename(columns={'All income recipients': 'avg_income_all', 'Male':'male', 'Female':'female', 'Under 30 years':'under_30',
       '30 ～ 34 years':'30_34yo', '35 ～ 39 years':'35_39yo', '40 ～ 44 years':'40_44yo','45 ～ 54 years':'45_54yo', 
       '55 ～ 64 years':'55_64yo', '65 years and over':'65plus','Primary school and below':'prim_school_below', 
       'Junior middle (vocational)':'jun_mid_voc','High school':'hs', 'Senior vocational school':'sen_voc', 
       'Junior college':'jun_coll','College and above':'coll_above'}, inplace=True)

In [17]:
# merge avg income sex
df['avg_income_sex'] = \
    np.where(df['sex'] == '1', df2['male'].values, 
             np.where(df['sex'] == '2', int(df2['female'].values), df['sex']))


In [18]:
# merge avg income age
age_cols = ['under_30', '30_34yo', '35_39yo','40_44yo', '45_54yo', '55_64yo', '65plus']

# np.select conditions
age_conditions = [
    (df['age'] < 30),
    (df['age'] >= 30) & (df['age'] <= 34),
    (df['age'] >= 35) & (df['age'] <= 39),
    (df['age'] >= 40) & (df['age'] <= 44),
    (df['age'] >= 45) & (df['age'] <= 54),
    (df['age'] >= 55) & (df['age'] <= 64),
    (df['age'] >= 65)
]

#np.select values
age_values = [
    df2['under_30'].values, 
    df2['30_34yo'].values, 
    df2['35_39yo'].values, 
    df2['40_44yo'].values, 
    df2['45_54yo'].values, 
    df2['55_64yo'].values, 
    df2['65plus'].values
]

# loop over age_cols to populate new column with np.select
for age in age_cols:
    df['avg_income_age'] = np.select(age_conditions, age_values)
    

In [19]:
# merge education columns 
edu_cols = ['prim_school_below','jun_mid_voc', 'hs', 'sen_voc', 'jun_coll', 'coll_above']

# conditions
edu_conditions =[
    (df['education'] == '0'),
    (df['education'] == '1'),
    (df['education'] == '2'),
    (df['education'] == '3'),
    (df['education'] == '4'),
    (df['education'] == '5'),
    (df['education'] == '6')
]

# values
edu_values = [
    df2[['sen_voc', 'jun_mid_voc', 'prim_school_below']].values.mean(), 
    df2['coll_above'].values,
    df2['jun_coll'].values,
    df2['hs'].values,
    df2[['sen_voc', 'jun_mid_voc', 'prim_school_below']].values.mean(),
    df2[['sen_voc', 'jun_mid_voc', 'prim_school_below']].values.mean(),
    df2[['sen_voc', 'jun_mid_voc', 'prim_school_below']].values.mean() 
]
    
for edu in edu_cols:
    df['avg_income_edu'] = np.select(edu_conditions, edu_values)
    

In [20]:
# create bill sum column

df['bill_sum'] = df[['bill_amt1', 'bill_amt2','bill_amt3', 'bill_amt4', 'bill_amt5', 'bill_amt6']].sum(axis=1)


In [21]:
# create pay sum column

df['pay_sum'] = df[['pay_amt1','pay_amt2', 'pay_amt3', 'pay_amt4', 'pay_amt5', 'pay_amt6']].sum(axis=1)


In [22]:
# subtract pay sum vs bill sum

df['bill_diff_pay'] = df['bill_sum'] - df['pay_sum']


In [23]:
# create a column that takes bill sum minus pay sum and subtracts from limit to create an available credit col

df['bill_diff_pay_min_limit'] = df['limit_bal'] - df['bill_diff_pay']


In [24]:
# create a general avg income column by avg together all the other avg income cols

df['avg_income'] = df[['avg_income_sex', 'avg_income_age','avg_income_edu']].mean(axis=1)


In [25]:
# create a debt to income ratio column
# divide avg income by 2 because its yearly, and sum of bills is for 6 month period

df['debt_to_income'] = df['bill_sum'] / (df['avg_income'] / 2)


In [26]:
# create debt to income ratio over 75% percentile

df['dti_75'] = np.where(df['debt_to_income'] > 1.29, 1, 0)


In [27]:
# create column that is a bool for married with university degree

df['married_univ'] = np.where((df['marriage'] == '1') & (df['education'] == '2'),1,0)


In [28]:
# dummy out categorical variables
cat_list = ['sex', 'education', 'marriage', 'pay_0', 'pay_2', 'pay_3', 'pay_4','pay_5', 'pay_6']

dummies = pd.get_dummies(df[cat_list], prefix=cat_list, drop_first=True)

df = df.drop(cat_list, axis=1)

df = pd.concat([df, dummies], axis=1)


In [29]:
# select features for final df
X = df[['limit_bal', 'age', 'bill_amt1', 'bill_amt2', 'bill_amt4', 'bill_amt5',
       'bill_amt6', 'pay_amt1', 'pay_amt2', 'pay_amt3', 'pay_amt6',
       'avg_income_age', 'avg_income_edu', 'pay_sum', 'bill_diff_pay',
       'bill_diff_pay_min_limit', 'avg_income', 'education_1', 'education_2',
       'education_3', 'marriage_1', 'marriage_2', 'marriage_3', 'pay_0_-2',
       'pay_0_0', 'pay_0_1', 'pay_0_2', 'pay_0_3', 'pay_0_4', 'pay_0_5',
       'pay_0_6', 'pay_0_7', 'pay_0_8', 'pay_2_-2', 'pay_2_0', 'pay_2_1',
       'pay_2_2', 'pay_2_4', 'pay_2_5', 'pay_2_6', 'pay_3_-2', 'pay_3_0',
       'pay_3_1', 'pay_3_2', 'pay_3_3', 'pay_3_5', 'pay_3_6', 'pay_3_8',
       'pay_4_-2', 'pay_4_1', 'pay_4_2', 'pay_4_5', 'pay_4_6', 'pay_4_7',
       'pay_4_8', 'pay_5_0', 'pay_5_2', 'pay_5_3', 'pay_5_4', 'pay_5_6',
       'pay_5_7', 'pay_6_0', 'pay_6_2', 'pay_6_3', 'pay_6_4', 'pay_6_7',
       'pay_6_8']]

In [30]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7501 entries, 5501 to 10111
Data columns (total 67 columns):
limit_bal                  7501 non-null float64
age                        7501 non-null int64
bill_amt1                  7501 non-null float64
bill_amt2                  7501 non-null float64
bill_amt4                  7501 non-null float64
bill_amt5                  7501 non-null float64
bill_amt6                  7501 non-null float64
pay_amt1                   7501 non-null float64
pay_amt2                   7501 non-null float64
pay_amt3                   7501 non-null float64
pay_amt6                   7501 non-null float64
avg_income_age             7501 non-null object
avg_income_edu             7501 non-null object
pay_sum                    7501 non-null float64
bill_diff_pay              7501 non-null float64
bill_diff_pay_min_limit    7501 non-null float64
avg_income                 7501 non-null float64
education_1                7501 non-null uint8
education_2  

In [31]:
# scale 
scaled_df = final_scaler.transform(X)


In [32]:
# predict on holdout data
final_answers = final_model.predict(scaled_df)


In [33]:
final_answers_df = pd.DataFrame(final_answers, columns=['holdout_pred'])


In [34]:
final_answers_df['holdout_pred'].value_counts(
)

0    4678
1    2823
Name: holdout_pred, dtype: int64

In [35]:
# export to csv
final_answers_df.to_csv('./data/preds_justin_williams.csv')
